In [ ]:
import os
import warnings
import sys

import pandas as pd
import numpy as np
from itertools import cycle
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import lasso_path, enet_path
from sklearn import datasets

In [ ]:
diabetes = datasets.load_diabetes()
X = diabetes.data
y = diabetes.target

In [ ]:
Y = np.array([y]).transpose()
d = np.concatenate((X, Y), axis=1)
cols = diabetes.feature_names + ["progression"]
data = pd.DataFrame(d, columns=cols)

In [ ]:
import newron
import newron.sklearn

In [ ]:
remote_server_uri = SERVER_URI # set to your server URI
newron.set_tracking_uri(remote_server_uri)  # or set the MLFLOW_TRACKING_URI in the env
exp_name = "sklearnExampleDiabetesElasticNet" # set your experiment name
newron.set_experiment(exp_name)

In [ ]:
# Evaluate metrics
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [ ]:
# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

# The predicted column is "progression" which is a quantitative measure of disease progression one year after baseline
train_x = train.drop(["progression"], axis=1)
test_x = test.drop(["progression"], axis=1)
train_y = train[["progression"]]
test_y = test[["progression"]]

alpha = 0.8
l1_ratio = 0.8

# Run ElasticNet
lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
lr.fit(train_x, train_y)
predicted_qualities = lr.predict(test_x)
(rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

# Print out ElasticNet model metrics
print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
print("  RMSE: %s" % rmse)
print("  MAE: %s" % mae)
print("  R2: %s" % r2)

# Log mlflow attributes for mlflow UI
newron.log_param("alpha", alpha)
newron.log_param("l1_ratio", l1_ratio)
newron.log_metric("rmse", rmse)
newron.log_metric("r2", r2)
newron.log_metric("mae", mae)
newron.sklearn.log_model(lr, "model")

# Compute paths
eps = 5e-3  # the smaller it is the longer is the path

print("Computing regularization path using the elastic net.")
alphas_enet, coefs_enet, _ = enet_path(X, y, eps=eps, l1_ratio=l1_ratio)

# Display results
fig = plt.figure(1)
ax = plt.gca()

colors = cycle(["b", "r", "g", "c", "k"])
neg_log_alphas_enet = -np.log10(alphas_enet)
for coef_e, c in zip(coefs_enet, colors):
    l2 = plt.plot(neg_log_alphas_enet, coef_e, linestyle="--", c=c)

plt.xlabel("-Log(alpha)")
plt.ylabel("coefficients")
title = "ElasticNet Path by alpha for l1_ratio = " + str(l1_ratio)
plt.title(title)
plt.axis("tight")

# Save figures
fig.savefig("ElasticNet-paths.png")

# Close plot
plt.close(fig)

# Log artifacts (output files)
newron.log_artifact("ElasticNet-paths.png")